CONFIGURAÇÃO DO AMBIENTE

In [ ]:
import subprocess
import sys

def upgrade_or_install(packages):
    """ Verifica se os pacotes estão instalados e os atualiza se necessário. """
    for package in packages:
        try:
            __import__(package)  # Tenta importar o pacote
            print(f"✅ O {package} já está instalado.")
            subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", package], check=True)
            print(f"🔄 O {package} foi atualizado com sucesso.")
        except ImportError:
            print(f"⚠️ O {package} não encontrado. iniciando a Instalação do pacote...")
            subprocess.run([sys.executable, "-m", "pip", "install", package], check=True)
            print(f"✅ O {package} instalado com sucesso.")

# Lista de pacotes necessários
packages = ["yfinance", "seaborn", "matplotlib", "pandas"]

# Verificar e instalar/atualizar pacotes
upgrade_or_install(packages)


MÓDULO PRINCIPAL COM LISTAS DE ATIVOS E FUNÇÃO DE DIVIDENDOS

In [ ]:
import yfinance as yf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Configuração para visualização
sns.set_theme(style="whitegrid")

# Lista de ações brasileiras com o sufixo .SA
stocks = [
    "PETR4.SA", "PRIO3.SA",
    "BBAS3.SA", "BBDC3.SA", "BRSR6.SA", "ABCB4.SA", "SANB4.SA", #BANCOS
    "BBSE3.SA", "CXSE3.SA",
    "MRVE3.SA", "JHSF3.SA", "LAVV3.SA"
    "BRAP4.SA", "VALE3.SA", "CMIN3.SA", #MINERAÇÃO
    "CMIG4.SA", "TAEE4.SA", "TRPL4.SA"  #ENERGIA
    "GOAU4.SA",
    "GGBR3.SA",
    "RANI3.SA", "SUZB3.SA", "CSAN3.SA",
    "WEGE3.SA", "LEVE3.SA"
    "TIMS3.SA", #BENS INDUSTRIAIS
    "MGLU3.SA" #VAREJO
]
print(f'O estudo envolve {len(stocks)} ações em diferentes setores da economia brasileira.')

# Baixar dados históricos
data = yf.download(stocks, start="2023-01-01", end="2024-01-01")["Close"]

# Remover colunas com valores NaN (se houver)
data = data.dropna(axis=1, how='all')

# Normalizar os preços para comparar desempenhos
desempenho = data / data.iloc[0]

def verificar_dividendos(acoes):
    """ Verifica e imprime os dividendos pagos por um conjunto de ações. """
    print("\n🔍 **Coletando dados sobre o pagamento de dividendos:**")
    print("\n Aguarde um momento...\n")

    for stock in acoes:
        acao = yf.Ticker(stock)
        dividendos = acao.dividends.loc["2024-01-01":"2025-01-01"]  # Dividendos no último ano
        total_dividendos = dividendos.sum()

        if total_dividendos > 0:
            print(f"✅ A {stock} pagou dividendos, no total de R$ {total_dividendos:.2f} no perído analisado.")
        else:
            print(f"❌Infelizmente, a {stock} não pagou dividendos no período analisado.")

COMPARATIVO DE PERFORMANCE DAS AÇÕES COM RANKING E GRÁFICOS, DIVIDINDO EM 3 GRUPOS

In [ ]:
# Chamando as funções para visualizar os gráficos na ordem correta
def upside_stock():
    """ Gera um gráfico das 5 ações com melhor desempenho e verifica dividendos. """
    top10 = desempenho.iloc[-1].nlargest(10).index  # Seleciona as 5 melhores ações
    top10 = list(top10)
    desempenho[top10].plot(figsize=(15, 6), title=f'GRÁFICO DAS {len(top10)} AÇÕES COM MELHOR DESEMPENHO NO PERÍODO', lw=2)
    plt.ylabel('Desempenho Normalizado')
    plt.xlabel('DATA')
    plt.legend(title='Ações', loc='upper left', fontsize='small')
    plt.show()

    verificar_dividendos(top10)
    return set(top10)

def downside_stock():
    """ Gera um gráfico das 5 ações com pior desempenho e verifica dividendos. """
    bottom10 = desempenho.iloc[-1].nsmallest(10).index  # Seleciona as 5 piores ações
    bottom10 = list(bottom10)
    desempenho[bottom10].plot(figsize=(15, 6), title=f'GRÁFICO DAS {len(bottom10)} AÇÕES COM PIOR DESEMPENHO NO PERÍODO', lw=2)
    plt.ylabel('Desempenho Normalizado')
    plt.xlabel('DATA')
    plt.legend(title='Ações', loc='upper left', fontsize='small')
    plt.show()

    verificar_dividendos(bottom10)
    return set(bottom10)

def half_stock(top10, bottom5):
    """ Gera um gráfico das ações que não estão entre as 5 melhores nem as 5 piores. """
    middle_stocks = set(desempenho.columns) - top10 - bottom10  # Remove as top 5 e bottom 5

    if middle_stocks:
        desempenho[list(middle_stocks)].plot(figsize=(15, 6), title='GRÁFICO DAS AÇÕES COM DESEMPENHO INTERMEDIÁRIO NO PERÍODO', lw=2)
        plt.ylabel('Desempenho Normalizado')
        plt.xlabel('DATA')
        plt.legend(title='Ações', loc='upper left', fontsize='small')
        plt.show()

        verificar_dividendos(middle_stocks)
    else:
        print("❌ Infelizmnete, não há ações para exibir.")

    return middle_stocks

CHAMADAS DAS FUNÇÕES SECUNDÁRIAS

In [ ]:
top10 = upside_stock()
bottom10 = downside_stock()
half = half_stock(top10, bottom10)